In [1]:
import numpy

In [3]:
# https://youtu.be/zN9ZINn7g24
"""
@author: DigitalSreeni

This file provides the VGG16 method to be applied on images to extract features.
These features can then be used for content based image retrieval.
Please note that Imagenet pre-trained weights will be loaded to VGG16
and the output from the final layer has a shape of 512 - our feature vector length.


"""

import numpy as np
from numpy import linalg as LA

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

#Understand the VGG16 model.
# model = VGG16(weights = 'imagenet',
#               input_shape = ((224, 224, 3)),
#               pooling = 'max',
#               include_top = False)
# model.summary()
#See how the final output gives us a vector oof size 512


class VGGNet:
    def __init__(self):
        # weights: 'imagenet'
        # pooling: 'max' or 'avg'
        # input_shape: (width, height, 3), width and height should >= 48
        self.input_shape = (296, 296, 3)
        self.weight = 'imagenet'
        self.pooling = 'max'
        self.model = VGG16(weights = self.weight, input_shape = (self.input_shape[0], self.input_shape[1], self.input_shape[2]), pooling = self.pooling, include_top = False)
        self.model.predict(np.zeros((1, 296, 296 , 3)))


    '''
    Use vgg16 model to extract features
    Output normalized feature vector
    '''
    def extract_feat(self, img):
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)
        feat = self.model.predict(img)
        norm_feat = feat[0]/LA.norm(feat[0])
        return norm_feat


In [16]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tensorflow.keras.preprocessing import image
from tensorflow.data import Dataset

# Initialize your VGGNet class
vgg = VGGNet()

def load_and_preprocess_image(image_path):
    img = Image.open(image_path)
    
    if img.mode != 'RGB':
        img = img.convert('RGB')
    
    img = img.resize((296, 296))  # Resize to match VGGNet input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

def process_batch(image_paths):
    batch_images = np.vstack([load_and_preprocess_image(img_path) for img_path in image_paths])
    batch_embeddings = vgg.model.predict(batch_images)
    batch_embeddings = [feat/LA.norm(feat) for feat in batch_embeddings]
    return batch_embeddings

# Define paths and initialize an empty DataFrame
imagenet_path = 'tiny-imagenet-200\\train'
selected_classes = {'lion': 'n02129165', 'bear': 'n02132136', 'tarantula': 'n01774750', 'golden retriever': 'n02099601'}

data = []

# Get a list of all image paths for the selected classes
all_image_paths = []
for class_name, class_code in selected_classes.items():
    class_path = os.path.join(imagenet_path, class_code)+'\images'
    if os.path.exists(class_path):
        image_paths = [os.path.join(class_path, img) for img in os.listdir(class_path)]
        all_image_paths.extend(image_paths)
    else:
        print(f"Class {class_name} not found in the dataset.")
# Process images in batches
batch_size = 32
for i in range(0, len(all_image_paths), batch_size):
    batch_paths = all_image_paths[i:i+batch_size]
    batch_embeddings = process_batch(batch_paths)
    
    for img_path, embedding in zip(batch_paths, batch_embeddings):
        image_name = os.path.basename(img_path)
        data.append({
            'image_name': image_name,
            'class_name': class_name,
            'embedding': embedding
        })
    
    # Optionally save progress after each batch
    if i % (batch_size * 100) == 0:  # Save every 100 batches
        print('saving patial')
        df = pd.DataFrame(data)
        df.to_csv('imagenet_embeddings_partial.csv', index=False)
        print('done saving partial')

# Convert final data to a DataFrame and save
df = pd.DataFrame(data)
df.to_csv('imagenet_embeddings.csv', index=False)


1/1 [==============================] - 6s 6s/step
saving partial
done saving partial
1/1 [==============================] - 10s 10s/step


KeyboardInterrupt: 

In [35]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

# Load the DataFrame
df = pd.read_csv('imagenet_embeddings.csv', header=0)

# Function to convert the string embeddings to NumPy arrays
def string_to_array(embedding_str):
    embedding_str = embedding_str.strip('[]')
    embedding_list = np.fromstring(embedding_str, sep=' ')
    return embedding_list

# Apply the conversion function to the 'embedding' column
df['embedding'] = df['embedding'].apply(string_to_array)

# Convert the list of embeddings into a 2D NumPy array (each row is an embedding)
embeddings_matrix = np.stack(df['embedding'].values)

# Compute the cosine similarity matrix
cosine_sim_matrix = 1 - cdist(embeddings_matrix, embeddings_matrix, metric='cosine')

# Optionally, convert the cosine similarity matrix to a DataFrame for better readability
cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=df['class_name'], columns=df['class_name'])




In [52]:
cosine_sim_df['tarantula'].loc['tarantula']

class_name,tarantula,tarantula,tarantula,tarantula,tarantula,tarantula,tarantula,tarantula,tarantula,tarantula,...,tarantula,tarantula,tarantula,tarantula,tarantula,tarantula,tarantula,tarantula,tarantula,tarantula
class_name,,,,,,,,,,,,,,,,,,,,,
tarantula,1.000000,0.673285,0.686922,0.710208,0.701993,0.671495,0.711839,0.675645,0.622477,0.611968,...,0.745794,0.672357,0.748322,0.648546,0.721817,0.719581,0.750351,0.720985,0.617027,0.733242
tarantula,0.673285,1.000000,0.720084,0.654756,0.628290,0.626029,0.671188,0.553494,0.599144,0.600740,...,0.663186,0.695152,0.677559,0.588674,0.711398,0.646778,0.718272,0.696753,0.569255,0.676147
tarantula,0.686922,0.720084,1.000000,0.707384,0.682693,0.721948,0.713780,0.670629,0.636366,0.696500,...,0.678618,0.734435,0.693928,0.585539,0.734333,0.712353,0.728110,0.733285,0.643574,0.708702
tarantula,0.710208,0.654756,0.707384,1.000000,0.727995,0.715018,0.696077,0.758832,0.679948,0.668072,...,0.745227,0.660075,0.700137,0.676023,0.710492,0.782275,0.741166,0.761651,0.704929,0.766907
tarantula,0.701993,0.628290,0.682693,0.727995,1.000000,0.680328,0.692569,0.711574,0.650067,0.607907,...,0.738984,0.663568,0.708432,0.686406,0.705313,0.744818,0.701486,0.727423,0.672215,0.800701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tarantula,0.719581,0.646778,0.712353,0.782275,0.744818,0.728913,0.710432,0.774820,0.747196,0.721452,...,0.720788,0.681664,0.679229,0.684848,0.731460,1.000000,0.726574,0.799195,0.759546,0.794887
tarantula,0.750351,0.718272,0.728110,0.741166,0.701486,0.666180,0.720069,0.663528,0.709524,0.692546,...,0.731632,0.747378,0.731297,0.638115,0.730255,0.726574,1.000000,0.733721,0.632384,0.742802
tarantula,0.720985,0.696753,0.733285,0.761651,0.727423,0.761238,0.735217,0.704414,0.751593,0.718545,...,0.712410,0.715281,0.704780,0.689960,0.751258,0.799195,0.733721,1.000000,0.732171,0.812458


In [6]:
#### Sort Validation Images to Classes ####
import os
import shutil

# Define paths
val_dir = 'tiny-imagenet-200/val'  # Adjust this path
val_img_dir = os.path.join(val_dir, 'images')
val_annotations = os.path.join(val_dir, 'val_annotations.txt')
dest_dir = 'tiny-imagenet-200/val_sorted'

# Create directories for each class
with open(val_annotations, 'r') as f:
    for line in f:
        parts = line.strip().split()
        img_file = parts[0]
        class_dir = parts[1]
        class_path = os.path.join(dest_dir, class_dir)
        if not os.path.exists(class_path):
            os.makedirs(class_path)

        # Move the image to the correct class directory
        img_src = os.path.join(val_img_dir, img_file)
        img_dest = os.path.join(class_path, img_file)
        shutil.move(img_src, img_dest)

print("Validation images sorted into class folders.")


Validation images sorted into class folders.
